# Notebook

## TODO:
* #odejmij wartość 1. bodźcda czasu + podziel przez 10000, aby mieć milisekundy
* #wartośc block end-block start
* #targety - regressor of no interest
* #możesz zbierać fragmenty w których pojawia się cue
* #podsumowania behawioralne


In [84]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [102]:
def read_file(filename):
    file = pd.read_csv(filename,"\t",skiprows=2)
    if np.any(file.columns.str.contains('Subject')):
        return file
    else:
        file = pd.read_csv(filename,"\t",skiprows=3)
        file = file.dropna(subset = ['Subject'])
        return file
        

data = read_file("6_runs_SIGHTED-6_runs_Fastloc_1back_sighted_run1.log")

def time_correction(data):
    """
    Corrects time in data from Presentation. 
    Reduce all Time column by time value of first event, 
    then divides by 10000 (to turn into ms).
    Returns whole DataFrame.
    """
    data['Time'] = (data['Time'] - data['Time'].iloc[0])/10000
    return data

def get_arg_block(data):
    """Get arg of blocks in data"""
    cond = ['block' in n.lower() for n in data['Code']]
    return (data
            .loc[cond]
            .index)

def get_list_endpoints(arg_data_part):
    """ 
    Get arg of starting and ending block, returns 
    list with shape [[arg_start,arg_end]...]
    """
    return [[arg_data_part[arg],arg_data_part[arg+1]] for \
            arg in range(0,len(arg_data_part),2)]

def present_target(start_time,end_time):
    """
    Check for the presence of the target.
    """
    return np.any((data
                   .loc[start_time:end_time,'Code']
                   .str.contains('target')))


def get_time_point_results(time_points):
    """
    Get results from data, for a given time_point
    """
    code_name = data.loc[time_points[0],'Code']
    start_time = data.loc[time_points[0],'Time']
    end_time = (data.loc[time_points[1],'Time']-start_time)
    target_val = present_target(time_points[0],time_points[1])
    return (code_name,start_time,end_time,target_val)

def single_block_results(endpoints):
    return pd.Series({'Code':endpoints[0],
                      'StartTime':endpoints[1],
                      'EndTime':endpoints[2],
                     'target':endpoints[3]})

def merge_blocks(previous_block,endpoints):
    """
    Merge previous block with new one.
    """
    tmp_series =single_block_results(endpoints)
    try:
        return previous_block.append(tmp_series,ignore_index=True)
    except:
        return pd.DataFrame().append(tmp_series,ignore_index=True)

data = time_correction(data)
chunk = get_arg_block(data)
endpoints = get_list_endpoints(chunk)        
end_block_times = map(get_time_point_results,endpoints)
reduce(merge_blocks,end_block_times)

# pd.value_counts(data['Stim Type'])

,Code,EndTime,StartTime,target
0,BLOCK_START_VIS_WORDS,8.0669,18.6550,0.0
1,BLOCK_START_VIS_PSEUDOWORDS,8.0670,31.7556,1.0
2,BLOCK_START_AUDIO_CONTROL,8.0003,45.3896,0.0
3,BLOCK_START_AUDIO_PSEUDOWORDS,8.0003,57.4235,1.0
4,BLOCK_START_AUDIO_WORDS,8.0003,68.9574,0.0
5,BLOCK_START_VIS_CONTROL,8.0669,85.0249,1.0
6,BLOCK_START_VIS_WORDS,8.0669,95.6254,0.0
7,BLOCK_START_VIS_PSEUDOWORDS,8.0669,107.7260,1.0
8,BLOCK_START_AUDIO_WORDS,8.0003,122.3600,1.0
9,BLOCK_START_AUDIO_PSEUDOWORDS,8.0003,133.8939,0.0
